# part 5  데이터 누락 처리

In [1]:
# 1. 누락 데이터 처리
#  -- 누락 데이터 확인
#  --  count()
#  -- value_counts()
#  -- isnull()  -> isnull.count()

In [2]:
import seaborn as sns

df = sns.load_dataset('titanic')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [4]:
# deck 컬럼의 값들의 개수 : value_counts()
df['deck'].value_counts(dropna=False)

NaN    688
C       59
B       47
D       33
E       32
A       15
F       13
G        4
Name: deck, dtype: int64

In [ ]:
# isnull() : 누락 데이터 개수 확인
print(df.head().isnull().sum(axis=0))
print(df.isnull().sum(axis=0))  # age : 177, deck : 688, embark_down : 2

In [12]:
df.columns

# 누락 데이터가 ?(500) 개 이상인 경우 컬럼을 삭제
df_1 = df.dropna(axis=1, thresh=500)
df_1.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'embark_town', 'alive',
       'alone'],
      dtype='object')

In [29]:
df_2 = df.dropna(subset=['deck','age'], how='any', axis=0)
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 184 entries, 1 to 889
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     184 non-null    int64   
 1   pclass       184 non-null    int64   
 2   sex          184 non-null    object  
 3   age          184 non-null    float64 
 4   sibsp        184 non-null    int64   
 5   parch        184 non-null    int64   
 6   fare         184 non-null    float64 
 7   embarked     182 non-null    object  
 8   class        184 non-null    category
 9   who          184 non-null    object  
 10  adult_male   184 non-null    bool    
 11  deck         184 non-null    category
 12  embark_town  182 non-null    object  
 13  alive        184 non-null    object  
 14  alone        184 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 18.4+ KB


In [26]:
# 누락 데이터 치환 : df.fillna(method='ffill') bfill, 값
# age 컬럼에 Nan 을 age의 평균값으로 치환
# 가장 빈번하게 나오는 값으로 치환
# 가장 큰 값으로 치환, 가장 작은 값으로 치환
# 이전 값으로 치환, 이후 나오는 값으로 치환 fillna(method='ffill' or 'bfill')

df_2 = df.copy()
print(df_2.age.isnull().value_counts())

# df['age'].mean(axis=0)  -> age의 평균으로 치환
# df_2['age'] = df_2['age'].fillna(df['age'].mean(axis=0))
df_2['age'].fillna(0, inplace=True)

df_2.age.isnull().value_counts()
df_2.age

False    714
True     177
Name: age, dtype: int64


0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
       ... 
886    27.0
887    19.0
888     0.0
889    26.0
890    32.0
Name: age, Length: 891, dtype: float64

In [ ]:
print(df['embark_town'][825:830])  # series 반환
# embark_town 의  Nan -> 이전 값으로 대체
df_2 = df.copy()
df_2['embark_town'].fillna(method='ffill', inplace=True)
df_2.embark_town[825:830]

In [39]:
print(df['embark_town'][825:831])  # series 반환
# embark_town 의  Nan -> 이후에 나오는 값으로 대체
df_2 = df.copy()
df_2['embark_town'].fillna(method='bfill', inplace=True)
df_2.embark_town[825:831]

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829            NaN
830      Cherbourg
Name: embark_town, dtype: object


825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829      Cherbourg
830      Cherbourg
Name: embark_town, dtype: object

In [36]:
# embark_town 에 Nan -> 가장 빈번하게 나오는 값으로 대체
most_freq = df['embark_town'].value_counts(dropna=True).idxmax()
df_2 = df.copy()
df_2['embark_town'].fillna(most_freq, inplace=True)
df_2.embark_town[825:830]

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829    Southampton
Name: embark_town, dtype: object

In [44]:
 df['embark_town'].value_counts().idxmax()

'Southampton'

In [47]:
# 누락 데이터가 Nan이 아닌 경우 : 특수 문자인 경우
# 특수 문자를 Nan으로 치환 후 Nan 처리
import pandas as pd
import numpy as np

df = pd.read_csv('./dataset/auto-mpg.csv', header=None)
df

df.columns =  ['mpg','cylinders','displacement','horsepower','weight',
             'acceleration','model year','origin','name']
df.info()

,0,1,2,3,4,5,6,7,8
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.00,2790.0,15.6,82,1,ford mustang gl
394,44.0,4,97.0,52.00,2130.0,24.6,82,2,vw pickup
395,32.0,4,135.0,84.00,2295.0,11.6,82,1,dodge rampage
396,28.0,4,120.0,79.00,2625.0,18.6,82,1,ford ranger


In [ ]:
# df['horsepower'].astype(float)  # error , 문자가 포함되어 있음
df['horsepower'].unique()  # '?' 문자가 발견 됨

In [53]:
# 1단계 : '?' 를 Nan으로 치환
df['horsepower'].replace('?', np.nan, inplace=True)  # 

# 2단계 : horsepower 의 값이 Nan 이면 해당 행을 삭제
df.dropna(subset='horsepower', axis=0, inplace=True)
df['horsepower'] = df['horsepower'].astype('float')

In [54]:
df['horsepower'].unique()

array([130., 165., 150., 140., 198., 220., 215., 225., 190., 170., 160.,
        95.,  97.,  85.,  88.,  46.,  87.,  90., 113., 200., 210., 193.,
       100., 105., 175., 153., 180., 110.,  72.,  86.,  70.,  76.,  65.,
        69.,  60.,  80.,  54., 208., 155., 112.,  92., 145., 137., 158.,
       167.,  94., 107., 230.,  49.,  75.,  91., 122.,  67.,  83.,  78.,
        52.,  61.,  93., 148., 129.,  96.,  71.,  98., 115.,  53.,  81.,
        79., 120., 152., 102., 108.,  68.,  58., 149.,  89.,  63.,  48.,
        66., 139., 103., 125., 133., 138., 135., 142.,  77.,  62., 132.,
        84.,  64.,  74., 116.,  82.])

In [56]:
# 중복 데이터 처리
# 중복 데이터를 갖는 데이터프레임 만들기
df = pd.DataFrame({'c1':['a', 'a', 'b', 'a', 'b'],
                  'c2':[1, 1, 1, 2, 2],
                  'c3':[1, 1, 2, 2, 2]})

df.duplicated()

0    False
1     True
2    False
3    False
4    False
dtype: bool

In [ ]:
# 'c2' 컬럼에 대해서 중복 체크
col_dup = df['c2'].duplicated()
col_dup

In [ ]:
# 중복 제이터 제거하는 함수 : drop_duplicates()
df2 = df.drop_duplicates()
df2

In [ ]:
# 중복 제이터 제거하는 함수 : drop_duplicates()
print(df)
df2 = df.drop_duplicates(subset=['c2','c3'])
df2

In [61]:
df2 = df.drop_duplicates(subset=['c1'])
df2

,c1,c2,c3
0,a,1,1
2,b,1,2


In [62]:
# 데이터 정제
# 1. 누락 데이터 처리 -> Nan, 특수문자
#     dropna(subset=[ ] , thresh, axis=0, inplace=True), 
#     fillna(값, method='ffill' or 'bfill')
# 2. 중복 데이터 처리
#    drop_duplicates(subset=[ ])
# 3. 데이터 표준화
#    -1. 단위 환산
#    -2. 자료형 변환 
# 4. 범주형 데이터 처리 : 카테고리형으로 변환
#    구간으로 데이터를 나누어서 처리
# 5. 더미 변수 : 카테고리 데이터인 경우 0와 1 로 값을 변환 시킴
# 6. train과 test 분리 -> 모델 학습

In [63]:
# 단위 환산
df = pd.read_csv('./dataset/auto-mpg.csv', header=None)

df.columns =  ['mpg','cylinders','displacement','horsepower','weight',
             'acceleration','model year','origin','name']
df.head(3)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite


In [65]:
# mpg(mile per gallon)  -> kpl  단위 환산 
df['kpl'] = ((df['mpg'] * 1.609) / 3.78).round(2)
df.head(3)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,kpl
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu,7.66
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,6.38
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite,7.66


In [71]:
# 자료형 변환 -> 데이터 타입 변환 , 문자를 숫자로, 숫자를 문자로
# df.dtypes
# df['horsepower'].dtype  -> object 를 float으로 형 변환
# df['horsepower'] = df['horsepower'].astype('float')

df['horsepower'].unique()  # '?' -> Nan으로 치환, Nan 처리
df['horsepower'].replace('?', np.nan, inplace=True)
df['horsepower'] = df['horsepower'].astype('float')

df['horsepower'].dtype 

dtype('float64')

In [77]:
# origin 컬럼의 정보 분석
print(df['origin'].dtype)
print(df['origin'].unique())   # 중복을 제거하고 유일한 값을 출력

# int 형을 범주형으로 변환
df['origin'] = df['origin'].astype('category')
print(df['origin'].dtype)
print(df['origin'].unique())

# 카테고리의 값을 문자형으로 변경 : 1->USA, 2->EU, 3->JPN
df['origin'].replace({1:'USA', 2:'EU', 3:'JPN'}, inplace=True)
print(df['origin'].unique())

category
[1, 3, 2]
Categories (3, int64): [1, 2, 3]
category
[1, 3, 2]
Categories (3, int64): [1, 2, 3]
['USA', 'JPN', 'EU']
Categories (3, object): ['USA', 'EU', 'JPN']


In [ ]:
# 범주형 자료의 데이터 타입을 'str'로 변경 가능
df['origin'] = df['origin'].astype('str')
print(df['origin'].dtype)

In [81]:
df['origin'].sample(3)

349    JPN
347    JPN
282    USA
Name: origin, dtype: object

In [86]:
# model year 컬럼을 분석 -> 카테고리형으로 변형
df['model year'].value_counts().info()

df['model year'] = df['model year'].astype('category')
df['model year'].dtype

<class 'pandas.core.series.Series'>
Int64Index: 13 entries, 73 to 74
Series name: model year
Non-Null Count  Dtype
--------------  -----
13 non-null     int64
dtypes: int64(1)
memory usage: 208.0 bytes


CategoricalDtype(categories=[70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82], ordered=False)

In [93]:
# 문자 또는 숫자를 카테고리형으로 변경 
# 원-핫 인코딩을 통해 자료 변형
# 연속형 자료를 범주형으로 변경
# 1. horsepower 컬럼을 3구간으로 나누어서 범주형으로 나눔
# 2. 각 구간별 경계선의 값이 필요함 : np.histogram(컬럼명, bins=구간수)
#    각 구간별 경계값과 구간에 소속된 데이터 갯수를 튜플로 반환
# np.histogram(df['horsepower'], bins = 3)

df['horsepower'].isnull().value_counts()  # Nan 검색
df.dropna(subset=['horsepower'], axis=0, inplace=True)  # Nan이 있는 행을 삭제
df['horsepower'].isnull().value_counts()

False    392
Name: horsepower, dtype: int64

In [101]:
(count, bin_dividers) = np.histogram(df['horsepower'], bins = 3)
print(bin_dividers)

bin_labels = ['저출력', '보통출력', '고출력']

# cut() 함수를 사용하여 각 값들을 bin에 할당
df['hp_bin'] = pd.cut(x=df['horsepower'],
                     bins = bin_dividers,
                     labels= bin_labels,
                     include_lowest= True)

df[['horsepower', 'hp_bin']].head(10)

[ 46.         107.33333333 168.66666667 230.        ]


,horsepower,hp_bin
0,130.0,보통출력
1,165.0,보통출력
2,150.0,보통출력
3,150.0,보통출력
4,140.0,보통출력
5,198.0,고출력
6,220.0,고출력
7,215.0,고출력
8,225.0,고출력
9,190.0,고출력


In [98]:
df['hp_bin'].dtype

CategoricalDtype(categories=['저출력', '보통출력', '고출력'], ordered=True)

In [100]:
# 더미 변수
pd.get_dummies(df['hp_bin'].head(10))

,저출력,보통출력,고출력
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
5,0,0,1
6,0,0,1
7,0,0,1
8,0,0,1
9,0,0,1


In [105]:
pd.get_dummies(df['origin'][10:30], prefix='origin')

,origin_EU,origin_JPN,origin_USA
10,0,0,1
11,0,0,1
12,0,0,1
13,0,0,1
14,0,1,0
15,0,0,1
16,0,0,1
17,0,0,1
18,0,1,0
19,1,0,0


In [117]:
titanic = sns.load_dataset('titanic')
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [119]:
titanic['alive'].sample(10) # object -> category 로 변경
titanic['alive'] = titanic['alive'].astype('category')

alive_dummy = pd.get_dummies(titanic['alive'], prefix='alive', drop_first=True)
pd.concat([titanic, alive_dummy], axis=1)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,alive_yes
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,0
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,1
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,1
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,1
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True,0
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True,1
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False,0
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True,1


In [ ]:
df1 = pd.DataFrame({'a1':[1,2,3,4],
                   'b1':[5,6,7,8]})

print(df1)

df2 = pd.DataFrame({'c1':[1,2,3,4],
                   'd1':[5,6,7,8]})
df2

In [131]:
pd.concat([df1,df2], axis=1)

,a1,b1,c1,d1
0,1,5,1,5
1,2,6,2,6
2,3,7,3,7
3,4,8,4,8


In [128]:
pd.concat([df1, df2], axis=1, join='outer' ) 

,a1,b1,c1,d1
a,1.0,5.0,NaN,NaN
b,2.0,6.0,NaN,NaN
c,3.0,7.0,NaN,NaN
d,4.0,8.0,NaN,NaN
0,NaN,NaN,1.0,5.0
1,NaN,NaN,2.0,6.0
2,NaN,NaN,3.0,7.0
3,NaN,NaN,4.0,8.0


In [123]:
pd.concat([df1, df2])   # 기본으로 행으로 결합
pd.concat([df1, df2], axis=1) 

,a1,b1,c1,d1
0,1,5,1,5
1,2,6,2,6
2,3,7,3,7
3,4,8,4,8
